<a href="https://colab.research.google.com/github/HosseinKarimi128/lumo/blob/main/lumo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive'
!source lumo/bin/activate


Mounted at /content/drive
/content/drive/MyDrive


# Packages

In [ ]:
# !pip install git+https://github.com/openai/whisper.git
!pip install transformers huggingsound pydub soundfile pyngrok flask jsonify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 52.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os, requests, torch, pydub, soundfile, transformers, spacy, re, logging
import scipy, IPython, pyngrok, flask, threading, nltk, collections, jsonify
import pandas as pd
import numpy as np

In [ ]:
!ngrok config add-authtoken 2Vyga8xLrSG3f6bom7aIUUogoly_bqJPjCZzZ7XVuYqx2rqS

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Load Data

In [ ]:
# data = pd.read_csv('/content/drive/MyDrive/lumo/response.csv', index_col=False)
data = pd.read_json('/content/drive/MyDrive/lumo/response.json')
data.drop(data.columns[0], axis=1,inplace=True)


In [ ]:
data.iloc[1].category[0]['name']

{'id': '67934a92-0443-11ee-be56-0242ac120002',
 'name': 'Prescription Refills',
 'color': 'green',
 'created_at': '2023-06-06T10:20:23.726716+00:00'}

In [ ]:
# def find_task(x):
#   if x: return x[0]['name']
data['task'] = data.category.apply(lambda x: x[0]['name'] if x!=[] else 'none')

In [ ]:
data.task = data.task.apply(lambda x: 'none' if x == None else x)
data.patient_phone = data.patient_phone.apply(lambda x: 'none' if x == None else x)

In [ ]:
data.head()

In [ ]:
data_for_patioent_name = data[data.patient_name != 'Maria']
print(len(data_for_patioent_name))
print(len(data))

# Transcript

In [ ]:
# load model and processor
processor = transformers.WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = transformers.WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
model.config.forced_decoder_ids = None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# samplerate, voice = scipy.io.wavfile.read(file_path)
# voice = voice.astype(np.float32)
# max_value = np.max(voice)
# voice /= max_value
# IPython.display.Audio(voice, rate=samplerate)

In [ ]:
def transcribe(filepath):
  file = pydub.AudioSegment.from_file(filepath)
  audio = file.set_frame_rate(16000)
  file.export("output.wav", format="wav")
  sample = pydub.AudioSegment.from_file("output.wav")
  sample = np.array(sample.get_array_of_samples())
  processor = transformers.WhisperProcessor.from_pretrained("openai/whisper-large-v2")
  model = transformers.WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
  model.config.forced_decoder_ids = None
  input_features = processor(sample, sampling_rate=16000, return_tensors="pt").input_features
  predicted_ids = model.generate(input_features)
  transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
  # transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
  return transcription[0]

# file_path = "/content/drive/MyDrive/lumo/voices/1t99xv6bbcdpypof3vzqa0ezl.wav"
# transcribe(pydub.AudioSegment.from_file(file_path))

# Models

In [ ]:
summarizer = transformers.pipeline("summarization", model="facebook/bart-large-cnn")
nlp = spacy.load("en_core_web_sm")
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
tokenizer = transformers.AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import joblib

def train_model(modelName, texts, labels):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(texts)
  clf = RandomForestClassifier()
  clf.fit(X, labels)
  joblib.dump(clf, '/content/drive/MyDrive/lumo/models/{}_random_forest_model.pkl'.format(modelName))
  joblib.dump(vectorizer, '/content/drive/MyDrive/lumo/models/{}_vectorizer.pkl'.format(modelName))

def predict(modelName,text):
  text = [text]
  clf = joblib.load('/content/drive/MyDrive/lumo/models/{}_random_forest_model.pkl'.format(modelName))
  vectorizer = joblib.load('/content/drive/MyDrive/lumo/models/{}_vectorizer.pkl'.format(modelName))
  return clf.predict(vectorizer.transform(text))

In [ ]:
train_model('title', data.transcript, data.title)
train_model('task', data.transcript, data.task)
train_model('phonNumber', data.transcript, data.patient_phone)

# predictors

In [ ]:
def get_title(text): return predict('title',text)
def get_summ(text): return summarizer(text, max_length=100, min_length=10, do_sample=False)[0]['summary_text']
def get_pationt(summ):
  doc = nlp(summ)
  return doc.ents[0].text
def get_doctor(summ):
  doc = nlp(summ)
  doctor = 'none'
  for i in range(len(doc)):
    if doc[i].text == "Dr.": doctor = doc[i].text+' '+doc[i+1].text
  return doctor
def get_task(summ):
  return predict('task',summ)
def get_phone_number(text):
  return predict('phonNumber',text)

In [ ]:
text = "Maria, this is Aladdin Waba. It's for Mary. The phone number is 647-344-7658. Mary is still in a lot of pain, and she was wondering if she could get a refill of the nanomaproxen and some Tylenol-3. Okay, thank you."
summ = get_summ(text)
task = get_task(summ)
phoneNumber = get_phone_number(text)
print('=========')
print(summ)
print('=========')
print(task)
print('=========')
print(phoneNumber)

Your max_length is set to 100, but your input_length is only 68. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


The phone number is 647-344-7658. Mary is still in a lot of pain. She was wondering if she could get a refill of the nanomaproxen and some Tylenol-3.
['Prescription Refills']
['647-344-7658']


# App

In [ ]:
def analyze(text):
  title = get_title(text)
  summ = get_summ(text)
  pationt = get_pationt(summ)
  doctor = get_doctor(summ)
  phoneNumber = get_phone_number(text)
  task = get_task(summ)
  return title,summ,pationt,doctor,phoneNumber,task

In [ ]:
from pyngrok import ngrok
!kill -9 "$(pgrep ngrok)"
os.environ["FLASK_ENV"] = "development"
app = flask.Flask(__name__)

public_url = ngrok.connect(5000,).public_url
app.config["BASE_URL"] = public_url
print(public_url)
file_handler = logging.FileHandler('app.log')
app.logger.addHandler(file_handler)
# ... Update inbound traffic via APIs to use the public-facing ngrok URL

home_t = '''
<form method="POST" action="/process" enctype="multipart/form-data">
  <input type="file" name="audio_data">
  <input type="submit">
</form>
'''
results_t='''
<style>
th {
  text-align: left;
}
</style>
<table>
  <tr>
    <th>Message:</th>
    <td>{{ results.message }}</td>
  </tr>

  <tr>
    <th>Title:</th>
    <td>{{ results.title }}</td>
  </tr>

  <tr>
    <th>Summary:</th>
    <td>{{ results.summ }}</td>
  </tr>

  <tr>
    <th>Patient:</th>
    <td>{{ results.pationt }}</td>
  </tr>

  <tr>
    <th>Doctor:</th>
    <td>{{ results.doctor }}</td>
  </tr>

  <tr>
    <th>Phone Number:</th>
    <td>{{ results.phoneNumber }}</td>
  </tr>

  <tr>
    <th>Task:</th>
    <td>{{ results.task }}</td>
  </tr>
</table>
'''

@app.route('/')
def home():
  app.logger.info('==============================')
  return home_t

def logger(msg): return jsonify({"txt": msg})

@app.route('/process', methods=['POST'])
def process():
  audio_file = flask.request.files['audio_data']
  audio_file.save('/content/drive/MyDrive/lumo/file.wav')
  message = transcribe('/content/drive/MyDrive/lumo/file.wav')
  title,summ,pationt,doctor,phoneNumber,task = analyze(message)
  results = {
      'message':message,
      'title':title,
      'summ':summ,
      'pationt':pationt,
      'doctor':doctor,
      'phoneNumber':phoneNumber,
      'task':task
      }

  return flask.render_template_string(results_t, results=results)
# Start the Flask server in a new thread
thread = threading.Thread(target=app.run, kwargs={"use_reloader": False})

/bin/bash: line 1: kill: `': not a pid or valid job spec


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


https://340a-34-67-171-216.ngrok-free.app


In [ ]:
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


In [ ]:
filename = '4wbwflk0u43jxpk9nl5bs2z4d.wav'
record = data[data['audio_url']==filename].iloc[0]
record.category

[{'id': '7f9ed6c4-0443-11ee-be56-0242ac120002',
  'name': 'Referral Requests',
  'color': 'purple',
  'created_at': '2023-06-06T10:20:23.726716+00:00'}]

In [ ]:
record.category

In [ ]:
# !fuser -k 5000/tcp

In [ ]:


# Sample messages
messages = list(data.transcript)
# Sample titles corresponding to messages
titles = list(data.title)

# Vectorize messages using CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(messages)

# Train a RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X, titles)


Predicted Title: Call back request for immunization software and blood work


In [ ]:
id = 525
print('\n')
print('message:', messages[id], '\n')
print('titles:', titles[id], '\n')

new_message = [messages[id]]
# new_message = ['hi my name is hossein, i want to set an appiontment for next month with dr aminian. Id appriciate if you could call me back, thank']

# Vectorize the new message
new_message_vectorized = vectorizer.transform(new_message)

# Predict the title
predicted_title = clf.predict(new_message_vectorized)

print("Predicted Title:", predicted_title[0], '\n')



message: Hi, this is Mehrun, Mehrun Ali. I was seeing Dr. Ramanian at Runnymede Clinic. I have registered her in her new office but have not seen her. I would like to make the appointment with her at a convenient time. Please call me to get the appointment from Dr. Ramanian. Thank you. 

titles: Appointment request 

Predicted Title: Appointment Scheduling 



# legacy